In [1]:
# !pip install gensim 
# !pip install pymorphy2
# !pip install summa
# !pip install python-rake
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()
from pymorphy2.tokenizers import simple_word_tokenize
from gensim.summarization import keywords as kw
from summa import keywords
import string
import re
import RAKE
import nltk
import os
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
rake = RAKE.Rake(stop)
from nltk.parse import DependencyGraph

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


датасет 500N-KPCrowd-v1.1 брала здесь: https://github.com/LIAAD/KeywordExtractor-Datasets/tree/master/datasets

в предоставляемом архиве тексты лежат в папке docsutf8, ключевые слова к ним находятся в папке keys. Файлы с текстами и соответствующими ключевыми совами имеют одинаковые называния. 
Ключевые слова для каждого текста записаны в отдельный файл через \n
ключевые слова могут включать в себя именную группу или клаузу

для составления своего мини-корпуса я взяла оттуда 15 текстов на три разныех темы: бизнесс, мода, исскуство

In [296]:
!C:\Users\User\Desktop\avtob\avtobreya\udpipe-1.2.0-bin\bin-win64\udpipe --input horizontal --output conllu \
--tokenize --tag --parse \
C:\Users\User\Desktop\avtob\avtobreya\english-ewt-ud-2.5-191206.udpipe \
< C:/Users/User/Desktop/avtob/avtobreya/key_words/500N-KPCrowd-v1.1/mini-corpus/text/fashion-20910434.txt > mini-corpus\synt_models_en\fashion-20910434.conllu
# записала 15 моделей для каждоого текста в папку \synt_models

Loading UDPipe model: done.


скачиваю тексты, создаю корпус вида corpus(text, key, my_key)

In [627]:
corpus = []

cwd = os.getcwd().replace('\\', '/')
path_text = cwd + '/mini-corpus/text/'
path_key = cwd + '/mini-corpus/keys/'
path_my_key = cwd + '/mini-corpus/my_keys/'
path_model = cwd + '/mini-corpus/synt_models_en/'

dir_texts= os.listdir(path_text)

for file in dir_texts:
    base=os.path.basename(file)
#     print(path_text + os.path.splitext(base)[0])
    with open(path_text + os.path.splitext(base)[0] + '.txt', 'r', encoding = 'utf-8') as f:
        text = f.read()
    with open(path_key + os.path.splitext(base)[0] + '.key', 'r', encoding = 'utf-8') as f:
        key = f.read().split('\n')
    with open(path_my_key + os.path.splitext(base)[0] + '.key', 'r', encoding = 'utf-8') as f:
        my_key = f.read().split('\n')
          
    corpus.append((text, key, my_key))

нахожу пересечение своей разметки и уже готовой. это будет эталоном

In [638]:
def create_etalon():
    etal = [x[1] for x in corpus]
    my_etal = [x[2] for x in corpus]

    etalon = []
    for ind, et in enumerate(etal):

        etalon_set = list(set(et) & set(my_etal[ind]))
        etalon.append([x.lower() for x in etalon_set])
        
    return etalon


In [298]:
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form)
    return ' '.join(lemmas)

разные способы выделения ключевых слов

In [523]:
def create_corpus_auto_keys(corpus):
    list_ra = []
    list_TextRank = []
    list_summa = []
    for text in corpus:
        text = normalize_text(text[0])
        
        ra = rake.run(text, maxWords=3, minFrequency=2)
        ra = [r[0] for r in ra]
        
        TextRank = kw(text, pos_filter=[], scores=True)
        TextRank = [t[0] for t in TextRank]
        
        summa = keywords.keywords(text, additional_stopwords=stop, scores=True)
        summa = [s[0] for s in summa]
        
        list_ra.append(ra)
        list_TextRank.append(TextRank)
        list_summa.append(summa)

    return list_ra, list_TextRank, list_summa

создаю свой способ поиска ключевых слов

In [521]:
_FILTER_RELS = ['punct', 'conj', 'parataxis']
def get_subtree(nodes, node):
    if not nodes[node]['deps']:
        return [node]
    else:
        return [node] + [get_subtree(nodes, dep) for rel in nodes[node]['deps'] 
                         if rel not in _FILTER_RELS
                         for dep in nodes[node]['deps'][rel]]
    
    
def flatten(l):
    flat = []
    for el in l:
        if not isinstance(el, list):
            flat.append(el)
        else:
            flat += flatten(el)
    return flat

In [544]:
def find_key_words(t, synt_mor = False):
    result = []
    try:
        np_list = []
        g = DependencyGraph(t, top_relation_label='root')
        for n in g.nodes:
            if g.nodes[n]['ctag'] == 'NOUN' or g.nodes[n]['ctag'] == 'PROPN':
                np = list(sorted(flatten(get_subtree(g.nodes, n))))
                np_list.append(
                    ' '.join([g.nodes[i]['word'].lower() for i in np]))
        if synt_mor:
            list_names = find_adg_noun(g)
            np_list += [x.lower() for x in list_names]
                
    except ValueError:
        print('pass')
    return np_list

это фильтр для поиска именных групп типа ADJ + NOUN.
ключевые слова, найденные таким образом добавляются в конец списка, если 
функции create_key_words() передать аргумент with_my_rules = True

In [545]:
def find_adg_noun(g):
    nouns = [('', '', '')]
    ret = []
    for n in g.nodes:
        b = g.get_by_address(n)['word']
        d = g.get_by_address(n)['ctag']
        if  (d == 'PROPN' or d == 'NOUN') and nouns[-1][2] == 'ADJ':
            nouns[-1] = nouns[-1][0] + ' ' + b
            ret.append(nouns[-1])

        nouns.append((b, n, d))
    return ret


функция проходится по всем файлам и находит ключевые слова для каждого
на основе разметри udpipe

In [595]:
def create_key_words(with_my_rules = False):
    corpus_udpipe = []
    curr = os.getcwd()
    for name_file in os.listdir(path_model):
        trees = []

        with open(curr + '\\mini-corpus\\synt_models_en\\' + name_file, 'r', encoding = 'utf-8') as f:
            parsed_sents = f.read().split('\n\n')
            corpus_words_per_senr = []
            for i, sent in enumerate(parsed_sents):

                tree = [line for line in sent.split('\n') if line and line[0] != '#']
                result = find_key_words(tree, with_my_rules)
                corpus_words_per_senr += result
            corpus_udpipe.append(corpus_words_per_senr)
    return corpus_udpipe

считает метрики

In [677]:
def count_metrics(i, corpus):
    right_keys = list(set(etalon[i]) & set(x for x in corpus[i] if len(x.split(' ')) < 3))
    
    if right_keys == []:
        return 0, 0, 0
    else:
        all_keys_find = [x for x in corpus[i] if len(x.split(' ')) < 3]
        precision = len(right_keys) / len(all_keys_find)
        recall = len(right_keys) / len(etalon[i])
        f1 = 2*((precision*recall) / (precision + recall))
        return round(precision, 2), round(recall, 2), round(f1, 2)

In [678]:
import pandas as pd

In [679]:
df = pd.DataFrame(columns = ['мой способ', 'rake', 'TextRank', 'summa'])

In [680]:
etalon = create_etalon()
corpus_udpipe = create_key_words()
list_ra, list_TextRank, list_summa = create_corpus_auto_keys(corpus)

for i in range(len(etalon)):
    df.loc[i,'мой способ'] = count_metrics(i, corpus_udpipe)
    df.loc[i]['rake'] = count_metrics(i, list_ra)
    df.loc[i]['TextRank'] = count_metrics(i, list_TextRank)
    df.loc[i]['summa'] = count_metrics(i, list_summa)
    

без примерения фильтра

In [681]:
df

,мой способ,rake,TextRank,summa
0,"(0.1, 0.21, 0.14)","(0, 0, 0)","(0.24, 0.11, 0.15)","(0.25, 0.11, 0.15)"
1,"(0.19, 0.17, 0.18)","(0.67, 0.1, 0.17)","(0.23, 0.07, 0.11)","(0.4, 0.14, 0.21)"
2,"(0.08, 0.27, 0.13)","(0.31, 0.12, 0.17)","(0.47, 0.24, 0.32)","(0.43, 0.27, 0.33)"
3,"(0.18, 0.19, 0.19)","(0.67, 0.06, 0.1)","(0.25, 0.06, 0.09)","(0.38, 0.08, 0.14)"
4,"(0.06, 0.18, 0.09)","(0.11, 0.06, 0.08)","(0.06, 0.06, 0.06)","(0.06, 0.06, 0.06)"
5,"(0.03, 0.12, 0.04)","(0.1, 0.05, 0.07)","(0.07, 0.1, 0.09)","(0.08, 0.1, 0.09)"
6,"(0.12, 0.23, 0.15)","(0.09, 0.03, 0.04)","(0.37, 0.17, 0.24)","(0.33, 0.17, 0.23)"
7,"(0.12, 0.23, 0.16)","(0.5, 0.19, 0.28)","(0.31, 0.15, 0.21)","(0.31, 0.15, 0.21)"
8,"(0.07, 0.19, 0.1)","(0.18, 0.07, 0.1)","(0.23, 0.16, 0.19)","(0.23, 0.16, 0.19)"
9,"(0.13, 0.14, 0.14)","(0, 0, 0)","(0.17, 0.07, 0.1)","(0.29, 0.07, 0.11)"


In [682]:
etalon = create_etalon()
corpus_udpipe = create_key_words(with_my_rules = True)
list_ra, list_TextRank, list_summa = create_corpus_auto_keys(corpus)

for i in range(len(etalon)):
    df.loc[i,'мой способ'] = count_metrics(i, corpus_udpipe)
    df.loc[i]['rake'] = count_metrics(i, list_ra)
    df.loc[i]['TextRank'] = count_metrics(i, list_TextRank)
    df.loc[i]['summa'] = count_metrics(i, list_summa)
    

с применением фильтра

In [683]:
df

,мой способ,rake,TextRank,summa
0,"(0.09, 0.21, 0.13)","(0, 0, 0)","(0.24, 0.11, 0.15)","(0.25, 0.11, 0.15)"
1,"(0.17, 0.17, 0.17)","(0.67, 0.1, 0.17)","(0.23, 0.07, 0.11)","(0.4, 0.14, 0.21)"
2,"(0.08, 0.27, 0.12)","(0.31, 0.12, 0.17)","(0.47, 0.24, 0.32)","(0.43, 0.27, 0.33)"
3,"(0.17, 0.19, 0.18)","(0.67, 0.06, 0.1)","(0.25, 0.06, 0.09)","(0.38, 0.08, 0.14)"
4,"(0.05, 0.18, 0.08)","(0.11, 0.06, 0.08)","(0.06, 0.06, 0.06)","(0.06, 0.06, 0.06)"
5,"(0.02, 0.12, 0.04)","(0.1, 0.05, 0.07)","(0.07, 0.1, 0.09)","(0.08, 0.1, 0.09)"
6,"(0.1, 0.23, 0.14)","(0.09, 0.03, 0.04)","(0.37, 0.17, 0.24)","(0.33, 0.17, 0.23)"
7,"(0.11, 0.23, 0.15)","(0.5, 0.19, 0.28)","(0.31, 0.15, 0.21)","(0.31, 0.15, 0.21)"
8,"(0.06, 0.19, 0.09)","(0.18, 0.07, 0.1)","(0.23, 0.16, 0.19)","(0.23, 0.16, 0.19)"
9,"(0.11, 0.14, 0.13)","(0, 0, 0)","(0.17, 0.07, 0.1)","(0.29, 0.07, 0.11)"


не все метрики хорошо работают с именами собственными, часто имя выделяется отдельно от фамилии, несмотря на то, что эти два слова состоят в одной именной группе. Это решается простым добавлением дополнительного фильтра PROPN + PROPN

при разметке могут допускаться опечатки, необходимо резметку проверять спеллчекером

в разметке встречались длинные отрывки фраз, это плохо потому что такие примеры при значении maxWords <= 3 не отлавливатся, а если это значение увеличивать, то будет попадать много мусорных примеров и понизятся значения метрик